This Version references a static spreadsheet to pull data copied from the intake form

In [1]:
# Install packages as needed
#%pip install openpyxl
#%pip install pandas 
#%pip install plotly
#%pip install nbformat
#%pip install --upgrade ipython

In [2]:
#import libraries
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale

In [3]:
# Read Transaction Excel data into a Pandas dataframe
#file = r'C:\Users\DIM6\OneDrive - PGE\Documents\GitHub\davemccallme.github.io\Cust-Trxn-Channel-Charts\Trxn-Data-Sunburst.xlsx'  #uses rough trxn data
file = r'C:\Users\DIM6\OneDrive - PGE\Documents\GitHub\davemccallme.github.io\Cust-Trxn-Channel-Charts\Sunburstdataframe-tmp.xlsx' #uses rough trxn data
#file = r'C:\Users\DIM6\OneDrive - PGE\Documents\GitHub\davemccallme.github.io\Cust-Trxn-Channel-Charts\scratch\Sunburstdataframe copy.xlsx' #uses lastest data

df = pd.read_excel(file)
df['Value'] = pd.to_numeric(df['Value'])
print(df.head())

        Root Level               Level 1 Level 2    Value
0  Call Center IVR              Payments     NaN  4474000
1  Call Center IVR               Billing     NaN  1241000
2  Call Center IVR             Pay Plans     NaN   209000
3  Call Center IVR       Account Balance     NaN   205000
4  Call Center IVR  Start/Stop/ Transfer     NaN   337000


In [4]:
# Write dataframe to an Excel file
with pd.ExcelWriter('Sunburstdataframe-tmp.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sunburst', index=False)

In [5]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col="Value", threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col="Value", threshold=10)

print(hierarchy_dict)

106324215


In [6]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [7]:
plotly_data = make_plotly_sunburst(hierarchy_dict)
print(plotly_data)

{'label': '', 'value': 106324215}


In [8]:
# Write JSON data to a file
with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

In [9]:
#Error Check rows for data discrepencies
for i, row in df.iterrows():
    try:
        fig = px.sunburst(df.loc[[i]], path=["Root Level", "Level 1", "Level 2"], values='Value')
    except ValueError as e:
        print(e)

In [10]:
 #Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - IVR': 'deepskyblue','Call Center - Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Format a sunburst chart using Plotly Express
color_sequence = [color_map[key] for key in color_map]
fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value')

# Update the layout
fig.update_layout(
    title='Transaction Channel Volume 2022',
    font=dict(size=14),
    margin=dict(t=50, l=0, r=0, b=0),
    width=1400,
    height=900,
    sunburstcolorway=color_sequence,
)
fig.show()


In [11]:
# Export the chart as an HTML file
pyo.plot(fig, filename='Trxn_sunburst_chart.html', auto_open=True)

'Trxn_sunburst_chart.html'